In [1]:
import numpy
from sklearn.feature_extraction import DictVectorizer 
import h2o 
from h2o.estimators.kmeans import H2OKMeansEstimator 
import pandas 
from function import * 
pandas.options.display.max_rows=1000 

In [2]:
h2o.init()

Connecting to H2O server at http://localhost:54321... successful!


H2O cluster uptime:,1 day 14 hours 39 mins
H2O cluster version:,3.10.0.3
H2O cluster version age:,26 days
H2O cluster name:,H2O_from_python_I_20545h
H2O cluster total nodes:,1
H2O cluster free memory:,3.179 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster is healthy:,True
H2O cluster is locked:,True
H2O connection url:,http://localhost:54321


#Rooms Level 

In [3]:
collection = mongo_collection() 
records = collection.find({'$or': [{'type': 'apartments'}, {'type': 'hotel'}]})

In [4]:
features_room_level, y_room_level = rooms_records(records) 

In [5]:
vectorizer_room_level = DictVectorizer() 
X_room_level = vectorizer_room_level.fit_transform(features_room_level)

In [6]:
room_level = pandas.DataFrame(X_room_level.toarray(), columns=normalize_features(vectorizer_room_level.get_feature_names())) 
room_level['type'] = y_room_level 

In [7]:
room_level_water = h2o.H2OFrame.from_python(room_level.to_dict('list')) 

In [8]:
columns = room_level_water.columns.copy() 

In [9]:
kmeans_room_level = H2OKMeansEstimator(k=2)

In [10]:
xc = columns.copy()
xc.remove('_Size') 

In [11]:
kmeans_room_level.train(training_frame=room_level_water, x=xc)

#####Standartized Centroids 

In [12]:
centroids_room_level_std = pandas.DataFrame(kmeans_room_level.centers_std(), index = xc.copy()) 

In [13]:
if centroids_room_level_std.loc['type'][0] == 'apartments': 
    centroids_room_level_std.rename(columns={0: 'Apartments', 1: 'Hotels'}, inplace=True)
else: 
    centroids_room_level_std.rename(columns={0: 'Hotels', 1: 'Apartments'}, inplace=True)
centroids_room_level_std.drop('type', inplace=True)
centroids_room_level_std['Difference'] = centroids_room_level_std['Apartments'] - centroids_room_level_std['Hotels']
centroids_room_level_std.sort_values('Difference', ascending=False)

,Hotels,Apartments,Difference
_Stovetop,-0.330479,1.64162,1.9721
_Kitchenware,-0.320137,1.59025,1.91039
_Oven,-0.313428,1.55692,1.87035
_Kitchen,-0.292244,1.45169,1.74394
_Dining_area,-0.291555,1.44827,1.73983
_Toaster,-0.274714,1.36461,1.63933
_Dining_table,-0.25762,1.2797,1.53732
_Washing_Machine,-0.23656,1.17509,1.41165
_Sofa,-0.234872,1.1667,1.40158
_Electric_kettle,-0.225344,1.11938,1.34472


#####Centroids 

In [14]:
centroids_room_level = pandas.DataFrame(list(zip(*kmeans_room_level.centers())), index=xc.copy()) 
if centroids_room_level.loc['type'][0] == 'apartments': 
    centroids_room_level.rename(columns={0: 'Apartments', 1: 'Hotels'}, inplace=True)
else: 
    centroids_room_level.rename(columns={0: 'Hotels', 1: 'Apartments'}, inplace=True)
centroids_room_level.drop('type', inplace=True)
centroids_room_level['Difference'] = centroids_room_level['Apartments'] - centroids_room_level['Hotels']
centroids_room_level.sort_values('Difference', ascending=False) 
centroids_room_level.sort_values('Apartments', ascending=False) 

,Hotels,Apartments,Difference
_Refrigerator,0.429481,0.968271,0.53879
_Toilet,0.835251,0.92745,0.0921995
_Private_bathroom,0.853089,0.923499,0.0704098
_Bathroom,0.841303,0.923499,0.082196
_Kitchenware,0.082931,0.866825,0.783894
_Stovetop,0.0436682,0.779039,0.735371
_Heating,0.656775,0.775316,0.118541
_Microwave,0.282386,0.769651,0.487265
_Seating_Area,0.301433,0.751148,0.449715
_Dining_area,0.068335,0.737419,0.669084


#Hotel Level 

In [15]:
collection = mongo_collection() 
records = collection.find({'$or': [{'type': 'apartments'}, {'type': 'hotel'}]})

In [16]:
features_hotel_level, y_hotel_level = hotel_records(records) 

In [17]:
vectorizer_hotel_level = DictVectorizer() 
X_hotel_level = vectorizer_hotel_level.fit_transform(features_hotel_level)

In [18]:
hotel_level = pandas.DataFrame(X_hotel_level.toarray(), columns=normalize_features(vectorizer_hotel_level.get_feature_names())) 
hotel_level['type'] = y_hotel_level 

In [19]:
hotel_level_water = h2o.H2OFrame.from_python(hotel_level.to_dict('list')) 

In [20]:
columns_hotel_level = hotel_level_water.columns.copy() 

In [21]:
kmeans_hotel_level = H2OKMeansEstimator(k=2) 

In [22]:
kmeans_hotel_level.train(training_frame=hotel_level_water, x=columns_hotel_level.copy()) 

#####Standartized Centroids 


In [23]:
centroids_hotel_level_std = pandas.DataFrame(kmeans_hotel_level.centers_std(), index = columns_hotel_level.copy()) 

In [24]:
if centroids_hotel_level_std.loc['type'][0] == 'apartments': 
    centroids_hotel_level_std.rename(columns={0: 'Apartments', 1: 'Hotels'}, inplace=True)
else: 
    centroids_hotel_level_std.rename(columns={0: 'Hotels', 1: 'Apartments'}, inplace=True)
centroids_hotel_level_std.drop('type', inplace=True)
centroids_hotel_level_std['Difference'] = centroids_hotel_level_std['Apartments'] - centroids_hotel_level_std['Hotels']
centroids_hotel_level_std.sort_values('Difference', ascending=False) 

,Hotels,Apartments,Difference
_Stovetop,-0.565866,1.12029,1.68615
_Kitchen,-0.546162,1.08128,1.62744
_Kitchenware,-0.525618,1.0406,1.56622
_Oven,-0.51166,1.01297,1.52463
_Dining_area,-0.456868,0.904495,1.36136
_Washing_Machine,-0.446925,0.884811,1.33174
_Refrigerator,-0.438439,0.868011,1.30645
_Sofa,-0.438368,0.86787,1.30624
_Dishwasher,-0.420046,0.831597,1.25164
_Microwave,-0.410802,0.813296,1.2241


#####Centroids 

In [25]:
centroids_hotel_level = pandas.DataFrame(list(zip(*kmeans_hotel_level.centers())), index=columns_hotel_level.copy()) 
if centroids_hotel_level.loc['type'][0] == 'apartments': 
    centroids_hotel_level.rename(columns={0: 'Apartments', 1: 'Hotels'}, inplace=True)
else: 
    centroids_hotel_level.rename(columns={0: 'Hotels', 1: 'Apartments'}, inplace=True)
centroids_hotel_level.drop('type', inplace=True)
centroids_hotel_level['Difference'] = centroids_hotel_level['Apartments'] - centroids_hotel_level['Hotels']
centroids_hotel_level.sort_values('Difference', ascending=False)  

,Hotels,Apartments,Difference
_Stovetop,0.0342774,0.800494,0.766216
_Kitchen,0.0389299,0.774082,0.735152
_Kitchenware,0.0653597,0.788453,0.723093
_Refrigerator,0.287966,0.941128,0.653161
_Oven,0.011267,0.647819,0.636552
_Microwave,0.167267,0.756603,0.589336
_Dining_area,0.0491739,0.635418,0.586244
_Sofa,0.0541628,0.613528,0.559365
_Washing_Machine,0.0188904,0.548025,0.529134
_Dishwasher,0.0232346,0.511292,0.488057
